# FEFO Mastery
Building a fefo calculator from scratch

In this Jupyter Notebook we want to write a comprehensive script that allows us to tackle inventory management strategy called FEFO, First Expiry First Out.

## Dictionaries for Inventory and Demand

Let's start with the most basic situation, we have a dictionary of articles and quantities called "Inventory" and we want to iterate through it so to print the data it contains

In [ ]:
# Create a variable called `inventory` and store some articles and quantities in it
inventory = {
    "apple": 100,
    "banana": 50,
    "orange": 75
}
# Write a loop to print the article and its quantity.
# We'll use the items() method to access the dictionary..
for k, v in inventory.items():
    print(k, v)


Now we will build upon the `dictionary` object.
Tipically inventory data is stored in relational databases and its format is perfectly represented by that of a `list` of `dictionaries`. Each `dictionary` contains `keys` like 'article' or 'sku_code', 'quantity', 'description' and so on.

In [ ]:
# Build a dictionary with three basic data points: 'article', 'quantity' and 'expiration_date'
inventory = [
    {"article": "apple", "quantity": 100, "expiration_date": "2025-01-15"},
    {"article": "banana", "quantity": 50, "expiration_date": "2025-01-10"},
    {"article": "orange", "quantity": 75, "expiration_date": "2025-01-20"}
]

# Learn to iterate through the dictionary accessing each entry or row and thus each `key - value` pair
for entry in inventory:
    print(f"{entry['article']}: {entry['quantity']}, expires on {entry['expiration_date']}")

## Sorting List of Dictionaries

It is very important at this point that we step out of **Python** and think of the **logic** that we want to implement. Here it goes: 
1. we want to allocate existing inventory to a demand (we haven't yet dealt with demand, but we'll do so very soon) _in order of expiry date_ 
2. we want to dinamically return the remaining inventory after it has been allocated and consumed by production

Let's tackle 1. Ordering the inventory by expiration date. 
We will do so by using a lamda function to sort the inventory. We will also assign the sorted inventory to a new variable which will allow us to keep track of all steps of the way.

In [ ]:
# Sort Inventory by Expiration Date
sortinv = sorted(inventory, key=lambda expi: expi['expiration_date'])

print("Sorted Inventory: \n")
for entry in sortinv:
    print(f"{entry['article']}: {entry['quantity']}, expires on {entry['expiration_date']}")

We will now build the demand dataset with a very simple list of two items. 
We will keep the same exact structure of the Inventory, i.e. a list of dictionaries, with keys 'article', 'quantity' and a date that tells us when the article is needed by production.

In [6]:
# Build the demand dataset
demand = [
    {"article": "apple", "quantity": 120, "demand_date": "2025-01-12"},
    {"article": "banana", "quantity": 30, "demand_date": "2025-01-11"}
]

We will also want to order the demand by date, this is because we want to allocate the existing inventory to demand that is closer in time - for later demand lines we will have time to purchase more inventory!

In [ ]:
# Sort demand by expiration date
sortdem = sorted(demand, key=lambda demd: demd['demand_date'])

print("Sorted Demand: \n")
for requir in sortdem:
    print(f"{requir['article']}: {requir['quantity']}, requirement on {requir['demand_date']}")

## A For Loop to Dynamically Allocate Inventory to Demand lines

Now that we have both demand and inventory in a good shape, we want to tackle the second problem, i.e. allocating the inventory to the demand and returning the remaining datasets. 

As this is a dinamic and iterative process we will need to use a for loop. And this procedure will essentially do the following: 
- for each line of the demand, take the first line of the inventory with corresponding article (of course we fulfill demand with the correct parts)
- if the demand is larger than the inventory line, then subtract the inventory and reduce it to zero
- - thus search for any other lines in the inventory that can satisfy the remaining demand
- if the demand is less than the inventory, then make the demand zero (fulfill the demand) and store the remaining inventory 

For clarity purposes we will want to delete zero line quantities from the two datasets. 
Also, as before, for clarity purposes, we keep track of all steps of the way by deep-copying our datasets. 

### Version 1: The Basic Calculation

In [ ]:
import copy
calc_inventory = copy.deepcopy(sortinv)
calc_demand = copy.deepcopy(sortdem)

# Match demand with existing inventory
for d in calc_demand:
    for i in calc_inventory:
        if i['article'] == d['article']:
            if d['quantity'] >= i['quantity']:
                d['quantity'] -= i['quantity'] # demand - inventory
                i['quantity'] = 0 # thus save the remaining demand
            else:
                i['quantity'] -= d['quantity'] # inventory - demand
                d['quantity'] = 0 # then reduce the demand to zero
calc_demand = [d for d in calc_demand if d['quantity'] > 0]
calc_inventory = [i for i in calc_inventory if i['quantity'] > 0]
# Reproduce the remaining inventory
print("Remaining inventory:")
for i in calc_inventory:
    print(f"{i['article']}: {i['quantity']}")
# Reproduce the remaining demand
print("\nRemaining demand:")
for i in calc_demand:
    print(f"{i['article']}: {i['quantity']}")

### Version 2: Handling Expiring Inventory and Dated Demand

Once we have made sure that we are able to capture the real life scenario of allocating ordered inventory to ordered demand, we will tackle the next problem, i.e. checking for the additional condition that the inventory is not expired at the time of demand. Essentially, the case where `inventory['expiration_date'] < demand['demand_date']` represents a case where the is no available inventory. Thus we need to exclude the allocation for that case. 

We initialize our sets and want to trigger this behaviour: 
- apples: there is a demand of 120 pieces which cannot be met by the lot expiring on the 10th of January. So that quantity "remains" and is not allocated to production. 
- the 120 demand is satisfied using the second lot, expiring on the 30th of the same month.
- however the subsequent demand of 30 pieces falls short of inventory, (only 130 - 120 = 10 pcs are left)

In [22]:
# initialize sets
demand = [
    {"article": "apple", "quantity": 120, "demand_date": "2025-01-12"},
    {"article": "apple", "quantity": 30, "demand_date": "2025-01-15"}]
inventory = [
    {"article": "apple", "quantity": 50, "expiration_date": "2025-01-10"}, 
    {"article": "apple", "quantity": 130, "expiration_date": "2025-01-30"}, 
    {"article": "banana", "quantity": 50, "expiration_date": "2025-01-10"}]

# order by date
sortdem = sorted(demand, key=lambda demd: demd['demand_date'])
sortinv = sorted(inventory, key=lambda expi: expi['expiration_date'])

# copy for tracking
calc_inv_date = copy.deepcopy(sortinv)
calc_dem_date = copy.deepcopy(sortdem)

We add `and d['demand_date'] < i['expiration_date']` to the conditional. This will ensure proper handling of the expiration and demand dates. 

In [ ]:
# Match demand with existing inventory
for d in calc_dem_date:
    for i in calc_inv_date:

        if d['article'] == i['article'] and d['demand_date'] < i['expiration_date']: # if the demand happens prior to the expiration
            if d['quantity'] >= i['quantity']:
                d['quantity'] -= i['quantity'] 
                i['quantity'] = 0 
            else:
                i['quantity'] -= d['quantity'] 
                d['quantity'] = 0 
        # in any other case the demand is unmet (and remains) and the inventory is expired (and remains)
    
# delete zero quanity lines
calc_dem_date = [d for d in calc_dem_date if d['quantity'] > 0]
calc_inv_date = [i for i in calc_inv_date if i['quantity'] > 0]

# Reproduce the remaining inventory
print("Remaining inventory:")
for i in calc_inv_date:
    print(f"{i['article']}: {i['quantity']}")
# Reproduce the remaining demand
print("\nUnmet demand:")
for i in calc_dem_date:
    print(f"{i['article']}: {i['quantity']}")

### Version 3: Tracking the Inventory Changes Before and After the Allocation

In the next round of code we are going to build upon the working loop of before. What we want to achieve is more clarity about the starting quantities, the consumed quantities and the remaining quantities for each line of the inventory and each line of the demand sets. 

1. the first thing we need is to initialize the dictionaries adding to `'quantity'` representing the initial quantity:
- demand set: `met_demand` and `unmet_demand`. Since we are _before_ the loop the demand met is 0 and the unmet demand is equal the the demand quantity
- inventory set: `remaining_quantity` and `consumed_quantity`. Again, the remaining quantity is the whole, and the consumed is 0, for now. 
_De facto_ we can think of these as the **dynamic** quantities of the dataset, while the initial quantity as a static amount.

2. Now we are ready to enter the loop, therefore, without touching the initial quantities, we are going to increase and decrease only the dynamic quantityes. 

3. At the end of the loop we will `display` the data with the help of Pandas DataFrames - which help us understand the result visually much better than plain `print` statements.

In [49]:
import pandas as pd
# initialize sets
demand = [
    {"article": "apple", "quantity": 120, "demand_date": "2025-01-12"},     # met: 120, um: 0
    {"article": "apple", "quantity": 30, "demand_date": "2025-01-15"}]      # met: 10, um: 20
inventory = [
    {"article": "apple", "quantity": 50, "expiration_date": "2025-01-10"},  # c: 0, r: 50
    {"article": "apple", "quantity": 130, "expiration_date": "2025-01-30"}, # c: 130, r: 0
    {"article": "banana", "quantity": 50, "expiration_date": "2025-01-10"}] # c: 0, r: 50

# order by date
sortdem = sorted(demand, key=lambda demd: demd['demand_date'])
sortinv = sorted(inventory, key=lambda expi: expi['expiration_date'])

# copy for tracking
import copy
calc_inv_date = copy.deepcopy(sortinv)
calc_dem_date = copy.deepcopy(sortdem)

# expand sets 
for i in calc_inv_date:
    i['remaining_quantity'] = i['quantity']
    i['consumed_quantity'] = 0

for d in calc_dem_date:
    d['met_demand'] = 0
    d['unmet_demand'] = d['quantity']

# Match demand to inventory and update the dynamic quantities
for d in calc_dem_date:
   
    for i in calc_inv_date:

        if d['article'] == i['article'] and d['demand_date'] < i['expiration_date']: # if the demand happens prior to the expiration
            
            if d['unmet_demand'] >= i['remaining_quantity']:    # if the unmet > remaining      |   (if d > i)
                
                d['met_demand'] += i['remaining_quantity']      # d.met quantity -> increase by the remaining
                d['unmet_demand'] -= i['remaining_quantity']    # set the unmet demand to zero
                i['consumed_quantity'] += i['remaining_quantity']
                i['remaining_quantity'] = 0                     # set the remaining to zero (move it to met quantity)
                            
            else:                                               # if the remaining > unmet      |   (if i > d)
                d['met_demand'] += d['unmet_demand']            # demand met is the entire unmet demad
                i['remaining_quantity'] -= d['unmet_demand']      # inventory remaining decrease by d.met
                i['consumed_quantity'] += d['unmet_demand']       # inventory consumed is decreased by the met demand
                d['unmet_demand'] = 0
          
# Display the result with Pandas
import pandas as pd
inventory_calc = pd.DataFrame(calc_inv_date)
demand_calc = pd.DataFrame(calc_dem_date)
display(inventory_calc)
display(demand_calc)

,article,quantity,expiration_date,remaining_quantity,consumed_quantity
0,apple,50,2025-01-10,50,0
1,banana,50,2025-01-10,50,0
2,apple,130,2025-01-30,0,130


,article,quantity,demand_date,met_demand,unmet_demand
0,apple,120,2025-01-12,120,0
1,apple,30,2025-01-15,10,20


### Version 4: Optimising the Calculation

optimisation of the iterative work: only on the grouped articles

now we use the grouped inventory `article` to process the allocation

In [27]:
import pandas as pd
# initialize sets
demand = [
    {"article": "apple", "quantity": 120, "demand_date": "2025-01-12"},     # met: 120, um: 0
    {"article": "apple", "quantity": 30, "demand_date": "2025-01-15"},      # met: 10, um: 20
    {"article": "kiwi", "quantity": 30, "demand_date": "2025-01-15"}]
inventory = [
    {"article": "apple", "quantity": 50, "expiration_date": "2025-01-10"},  # c: 0, r: 50
    {"article": "apple", "quantity": 130, "expiration_date": "2025-01-30"}, # c: 130, r: 0
    {"article": "banana", "quantity": 50, "expiration_date": "2025-01-10"}, # c: 0, r: 50
    {"article": "kiwi", "quantity": 25, "expiration_date": "2025-01-03"},
    {"article": "pineapple", "quantity": 2, "expiration_date": "2025-01-03"},]

# order by date
sortdem = sorted(demand, key=lambda demd: demd['demand_date'])
sortinv = sorted(inventory, key=lambda expi: expi['expiration_date'])

# copy for tracking
import copy
calc_inv_date = copy.deepcopy(sortinv)
calc_dem_date = copy.deepcopy(sortdem)

# expand sets 
for i in calc_inv_date:
    i['remaining_quantity'] = i['quantity']
    i['consumed_quantity'] = 0

for d in calc_dem_date:
    d['met_demand'] = 0
    d['unmet_demand'] = d['quantity']

# Create a dictionary of inventory skus
articles = {}
for item in calc_inv_date:
    sku = item['article']
    if sku not in articles: 
        articles[sku] = []              # add the sku and assign it an empty list
        articles[sku].append(item)         # append the row to the list within the sku key
    else:
        articles[sku].append(item)         # append as before

# Match demand to inventory and update the dynamic quantities
# pull from articles
for d in calc_dem_date:
    if d['article'] in articles:

        for i in articles[d['article']]:
            if d['demand_date'] < i['expiration_date']:
                
                if d['unmet_demand'] >= i['remaining_quantity']:    # if the unmet > remaining      |   (if d > i)
                    
                    d['met_demand'] += i['remaining_quantity']
                    d['unmet_demand'] -= i['remaining_quantity']
                    i['consumed_quantity'] += i['remaining_quantity']
                    i['remaining_quantity'] = 0
                                
                else:                                               # if the remaining > unmet      |   (if i > d)
                    d['met_demand'] += d['unmet_demand']
                    i['remaining_quantity'] -= d['unmet_demand']
                    i['consumed_quantity'] += d['unmet_demand']
                    d['unmet_demand'] = 0
            
import pandas as pd
inventory_calc = pd.DataFrame(calc_inv_date)
demand_calc = pd.DataFrame(calc_dem_date)
display(inventory_calc)
display(demand_calc)

,article,quantity,expiration_date,remaining_quantity,consumed_quantity
0,kiwi,25,2025-01-03,25,0
1,pineapple,2,2025-01-03,2,0
2,apple,50,2025-01-10,50,0
3,banana,50,2025-01-10,50,0
4,apple,130,2025-01-30,0,130


,article,quantity,demand_date,met_demand,unmet_demand
0,apple,120,2025-01-12,120,0
1,apple,30,2025-01-15,10,20
2,kiwi,30,2025-01-15,0,30


### Speed test
Let's now create large datasets to test the speed of each loop and compare them.

For simplicity, we are going to store 4 functions in the python file called `utils.py`:
1. the slow function with name `fefo_calc_opt`.
2. the optimised function with name `fefo_calc_slo`
3. `gen_inventory` a generator of random inventory lines that takes two arguments: `number_of_lines` and `range` for the randint method
4. `gen_demand` a generator of random demand, also with a parameter for `number_of_lines` and one for the `range`.

if you would like to see all these functions, feel free to take a look at the utils.py. But here We want to concentrate on the speed of the two models. 

Let's import the functions, generate **large** datasets

In [24]:
# import the 4 functions
from utils import gen_inventory, gen_demand, fefo_calc_opt, fefo_calc_slo

random_inventory = gen_inventory(10_000,1_000) # 100k lines and 1k random codes in total
random_demand = gen_demand(5_000,1_000) # 50K lines of demand with the same 1k random codes

In [25]:
# run the first model. 
fefo_calc_opt(random_demand, random_inventory) # 100K inventory and 50K demand lines -> 0.9 seconds

,article,quantity,expiration_date,remaining_quantity,consumed_quantity
0,a451,56777,2025-01-02,56777,0
1,a649,94033,2025-01-02,94033,0
2,a882,38735,2025-01-02,38735,0
3,a349,4923,2025-01-02,4923,0
4,a711,80012,2025-01-02,80012,0
...,...,...,...,...,...
9995,a753,51208,2026-01-02,0,51208
9996,a720,11682,2026-01-02,11682,0
9997,a555,71776,2026-01-02,29909,41867
9998,a695,55169,2026-01-02,55169,0


In [26]:
# run the second model
fefo_calc_slo(random_demand, random_inventory) # 100K inventory and 50K demand lines -> 4 minutes and 28 seconds

,article,quantity,expiration_date,remaining_quantity,consumed_quantity
0,a451,56777,2025-01-02,56777,0
1,a649,94033,2025-01-02,94033,0
2,a882,38735,2025-01-02,38735,0
3,a349,4923,2025-01-02,4923,0
4,a711,80012,2025-01-02,80012,0
...,...,...,...,...,...
9995,a753,51208,2026-01-02,0,51208
9996,a720,11682,2026-01-02,11682,0
9997,a555,71776,2026-01-02,29909,41867
9998,a695,55169,2026-01-02,55169,0


## Building Sets to Test all Cases

Now that we have stored our functions neatly we can proceed to test some cases. This is to make 100% sure that the model is behaving just like we wanted it to. 
Let's come up with cases where we are going to test that it returns the predicted result. For this we need to define a set of input lines and manually calculate the output, which then we are going to compare to our model's. 

### Set of test cases
#### 1. duplicate demand lines

In [2]:
from utils import fefo_calc_opt
import pandas as pd

demand = [
    {"article": "apple", "quantity": 30, "demand_date": "2025-01-12"},     
    {"article": "apple", "quantity": 30, "demand_date": "2025-01-12"},      
    {"article": "apple", "quantity": 30, "demand_date": "2025-01-12"}]
inventory = [
    {"article": "apple", "quantity": 70, "expiration_date": "2025-01-30"}]

d, i = fefo_calc_opt(demand, inventory)
display(d)
display(i)

# CHECK!
# All values of remaining, consumed, med and unmet quantities are correctly processed

,article,quantity,demand_date,met_demand,unmet_demand
0,apple,30,2025-01-12,30,0
1,apple,30,2025-01-12,30,0
2,apple,30,2025-01-12,10,20


,article,quantity,expiration_date,remaining_quantity,consumed_quantity
0,apple,70,2025-01-30,0,70


#### 2. demand that has no inventory  -> Shortages
#### 3. demand that has only expiring inventory -> Expiration

In [6]:
demand = [
    {"article": "kiwi", "quantity": 30, "demand_date": "2025-01-12"},     
    {"article": "apple", "quantity": 30, "demand_date": "2025-01-12"},      
    {"article": "apple", "quantity": 30, "demand_date": "2025-01-12"}]
inventory = [
    {"article": "apple", "quantity": 70, "expiration_date": "2025-01-01"}]

d, i = fefo_calc_opt(demand, inventory)
display(d)
display(i)

# CHECK!
# All values look good: all demand is unmet (either because of expiration or shortages) 

,article,quantity,demand_date,met_demand,unmet_demand
0,kiwi,30,2025-01-12,0,30
1,apple,30,2025-01-12,0,30
2,apple,30,2025-01-12,0,30


,article,quantity,expiration_date,remaining_quantity,consumed_quantity
0,apple,70,2025-01-01,70,0


One aspect that we notice here is that we don't have a unified view. For example, in this case 'kiwi' only appears
on the demand table, due to no inventory being there. At the same time that we have 70 apples expired we only know 
from the inventory table. For the purpose of making this a useful tool for ops managers or stakeholders, we should create a single source of truth to rely on. We will work on this later.

#### 4. Inventory that is split allocated to several demand lines
#### 5. Demand that is spit across inventory lines

In [5]:
demand = [
    {"article": "banana", "quantity": 30, "demand_date": "2025-01-12"},     
    {"article": "banana", "quantity": 30, "demand_date": "2025-01-15"},      
    {"article": "banana", "quantity": 30, "demand_date": "2025-01-18"}]
inventory = [
    {"article": "banana", "quantity": 10, "expiration_date": "2025-01-13"},
    {"article": "banana", "quantity": 60, "expiration_date": "2025-01-20"},
    {"article": "banana", "quantity": 20, "expiration_date": "2025-01-20"}]

d, i = fefo_calc_opt(demand, inventory)
display(d)
display(i)

# CHECK
# This also worked: we expected that all demand would be met leaving zero inventory, regardless of
# the lines of the two sets intermingling along the dynamic allocation process.


,article,quantity,demand_date,met_demand,unmet_demand
0,banana,30,2025-01-12,30,0
1,banana,30,2025-01-15,30,0
2,banana,30,2025-01-18,30,0


,article,quantity,expiration_date,remaining_quantity,consumed_quantity
0,banana,10,2025-01-13,0,10
1,banana,60,2025-01-20,0,60
2,banana,20,2025-01-20,0,20


Another thing we notice from the above exercise is that we don't exaclty know what line has been used from the inventory lines 2 and 3. This is because they are identical and, for computational purposes they don't make a difference. However, from anoperational point of view we could norm the allocation process according to some parameters, which we must add to the pipeline. For instance, `palled_id` or `location` or `inbound_date`. This would allow us to track more closely what exactly we want to happen in the production. 

#### 6. Past demand
Now, this is something we haven't tackled yet, so we know that the model won't be smart enough to return the correct result. But let's take a look:

In [10]:
from datetime import date
print(date.today())
demand = [
    {"article": "banana", "quantity": 30, "demand_date": "2025-01-12"}]
inventory = [
    {"article": "banana", "quantity": 10, "expiration_date": "2025-01-13"}]

d, i = fefo_calc_opt(demand, inventory)
display(d)

# The match checks, but it is strange that we could allocate past demand to expired inventory, even though the dates made sense from a relative point of view. 

2025-01-22


,article,quantity,demand_date,met_demand,unmet_demand
0,banana,30,2025-01-12,10,20


Takeaway here is that the model accepts relative date handling, but is not sensible to the actual date. 
I take this to be a feature, because, in fact, production could process past demand, in which case production is _late_. 
How I would tackle this problem is by building a data ingestion mechanism that runs the check: 'is_past', and returns `True` or `False`. In other words I would shift the problem to an earlier stage of the data pipeline: the importing of data from the database. Here we could select only future demand, e.g.
Apart from this check, I would leave the model capable of handling past demand freely.

### Tackling issues one by one

Though we have not yet found computational issues, we came up with a list of improvements for the output:
- output a single source of truth, merging inventory and demand information in one view.
- introduce additional identifiers for the physical goods (e.g. `pallet_id`) and bring this information through to the output
- Another idea: costify lines. 

Let's assume care about 'pallet id'. So we want to make sure that IDs with lower values go into production first, in case there is a duplicate line of same-article, same-exipration_date. 
The first thing we have to do after adding the `pal_id` key-value pair to our Inventory set, is to sort the inventory so that it takes the lines in the desired order. 

In [28]:
# Introduce Pallet IDs to the Inventory lines
inventory = [
    {"article": "banana", "quantity": 10, "expiration_date": "2025-01-30", "pal_id": 3},
    {"article": "banana", "quantity": 60, "expiration_date": "2025-01-20", "pal_id": 8},
    {"article": "banana", "quantity": 20, "expiration_date": "2025-01-20", "pal_id": 4}]

# Add a sorting parameter to the function 
calc_inv_date = sorted(inventory, key=lambda logic:(logic['expiration_date'], logic['pal_id']))

# Test the desired result is generated
for i in calc_inv_date:
    print(i)

# CHECK
# This is what we wanted: the first sorting method remains the expiration date (FEFO), and the second is the pallet ID.
# This is an arbitrary choice, of course, and it should be adapted to whichever is the desired logic in the unit. 

{'article': 'banana', 'quantity': 20, 'expiration_date': '2025-01-20', 'pal_id': 4}
{'article': 'banana', 'quantity': 60, 'expiration_date': '2025-01-20', 'pal_id': 8}
{'article': 'banana', 'quantity': 10, 'expiration_date': '2025-01-30', 'pal_id': 3}
